In [12]:
import nltk
from nltk.tokenize import word_tokenize
import gensim
from gensim.models.doc2vec import TaggedDocument
import numpy as np
import pandas as pd

In [3]:
df = pd.read_csv('data 2023.csv')

In [48]:
df

,program
0,"Koordinasi, Perencanaan, Supervisi dan Evaluas..."
1,Pembangunan Unit Sekolah Baru (USB)
2,Penambahan Ruang Kelas Sekolah
3,Pembangunan Ruang Serba Guna/Aula
4,"Pembangunan Sarana, Prasarana dan Utilitas Sek..."
...,...
167,Pendataan dan Pendaftaran Objek Pajak Daerah
168,Penetapan Wajib Pajak Daerah
169,Pelayanan dan Konsultasi Pajak Daerah
170,Penagihan Pajak Daerah


# tagging

In [4]:
# Fungsi untuk mengubah dokumen menjadi TaggedDocument
def tag_docs(docs):
    tagged_docs = []
    for i, doc in enumerate(docs):
        tagged_docs.append(TaggedDocument(words=doc.split(), tags=[i]))
    return tagged_docs

In [9]:
tagged_data = tag_docs(df['program'])

In [10]:
tagged_data

[TaggedDocument(words=['Koordinasi,', 'Perencanaan,', 'Supervisi', 'dan', 'Evaluasi', 'Layanan', 'di', 'Bidang', 'Pendidikan'], tags=[0]),
 TaggedDocument(words=['Pembangunan', 'Unit', 'Sekolah', 'Baru', '(USB)'], tags=[1]),
 TaggedDocument(words=['Penambahan', 'Ruang', 'Kelas', 'Sekolah'], tags=[2]),
 TaggedDocument(words=['Pembangunan', 'Ruang', 'Serba', 'Guna/Aula'], tags=[3]),
 TaggedDocument(words=['Pembangunan', 'Sarana,', 'Prasarana', 'dan', 'Utilitas', 'Sekolah'], tags=[4]),
 TaggedDocument(words=['Pembangunan', 'Kantin', 'Sekolah'], tags=[5]),
 TaggedDocument(words=['Pembangunan', 'Rumah', 'Dinas', 'Kepala', 'Sekolah,', 'Guru,', 'Penjaga', 'Sekolah'], tags=[6]),
 TaggedDocument(words=['Pembangunan', 'Ruang', 'Orientasi', 'dan', 'Mobilitas'], tags=[7]),
 TaggedDocument(words=['Pembangunan', 'Ruang', 'Bina', 'Persepsi', 'Bunyi', 'dan', 'Irama', 'untuk', 'Tunarungu', '(B)'], tags=[8]),
 TaggedDocument(words=['Pembangunan', 'Ruang', 'Bina', 'Diri', 'dan', 'Bina', 'Gerak', 'untuk',

In [25]:
def cosine(u, v):
    return np.dot(u, v) / (np.linalg.norm(u) * np.linalg.norm(v))

# Train model

In [26]:
# Melatih model Doc2Vec
model = gensim.models.Doc2Vec(vector_size=100, min_count=2, epochs=40)
model.build_vocab(tagged_data)
model.train(tagged_data, total_examples=model.corpus_count, epochs=model.epochs)

c:\users\hadi\appdata\local\programs\python\python38\lib\site-packages\gensim\models\base_any2vec.py:742: UserWarning: C extension not loaded, training will be slow. Install a C compiler and reinstall gensim for fast training.
  warnings.warn(


In [36]:
# Melakukan inferensi pada dokumen baru dari data CSV
new_doc_index = 0  # Indeks dokumen baru yang ingin diinferensi
new_doc = df['program'][new_doc_index]
inferred_vector = model.infer_vector(new_doc.split())

# Mendapatkan dokumen yang paling mirip berdasarkan vektor inferensi
similar_docs = model.docvecs.most_similar([inferred_vector])

# Mencetak dokumen yang paling mirip
for doc_id, similarity in similar_docs:
    print(f'Dokumen #{doc_id}: {df["program"][doc_id]}')
    print(f'Similaritas: {similarity}\n')

Dokumen #171: Pelaksanaan Koordinasi di Bidang Pendidikan Politik, Etika Budaya Politik, Peningkatan Demokrasi, Fasilitasi Kelembagaan Pemerintahan, Perwakilan dan Partai Politik, Pemilihan Umum/Pemilihan Umum Kepala Daerah, serta Pemantauan Situasi Politik di Daerah
Similaritas: 0.9995153546333313

Dokumen #103: Fasilitasi Penyediaan Sarana dan Prasarana Kelembagaan Lembaga KeMasyarakatan Desa/Kelurahan (RT, RW, PKK, Posyandu, LPM, dan Karang Taruna), Lembaga Adat Desa/Kelurahan dan Masyarakat Hukum Adat
Similaritas: 0.9994690418243408

Dokumen #101: Fasilitasi Penataan, Pemberdayaan dan Pendayagunaan Kelembagaan Lembaga Ke Masyarakatan Desa/Kelurahan (RT, RW, PKK, Posyandu, LPM, dan Karang Taruna), Lembaga Adat Desa/Kelurahan dan Masyarakat Hukum Adat
Similaritas: 0.9994239807128906

Dokumen #102: Peningkatan Kapasitas Kelembagaan Lembaga KeMasyarakatan Desa/Kelurahan (RT,RW, PKK, Posyandu, LPM, dan Karang Taruna), Lembaga Adat Desa/Kelurahan dan Masyarakat Hukum Adat)
Similaritas: 0

In [39]:
test_doc = word_tokenize("Daya saing Sumber Daya Manusia".lower())
test_doc_vector = model.infer_vector(test_doc)
similar_docs2 = model.docvecs.most_similar(positive = [test_doc_vector])

for doc_id, similarity in similar_docs2:
    print(f'Dokumen #{doc_id}: {df["program"][doc_id]}')
    print(f'Similaritas: {similarity}\n')

Dokumen #3: Pembangunan Ruang Serba Guna/Aula
Similaritas: 0.19642581045627594

Dokumen #85: Fasilitasi Kelembagaan Desa dan Desa Adat
Similaritas: 0.19177842140197754

Dokumen #110: Penyediaan Komponen Instalasi Listrik/Penerangan Bangunan Kantor
Similaritas: 0.18643856048583984

Dokumen #152: Penyediaan Peralatan Rumah Tangga
Similaritas: 0.18401774764060974

Dokumen #92: Fasilitasi Penyusunan, Perencanaan, Pelaksanaan dan Pengawasan Pembangunan Desa
Similaritas: 0.1790962517261505

Dokumen #10: Pembangunan Ruang Bina Pribadi dan Sosial untuk Tunalaras (E)
Similaritas: 0.17903079092502594

Dokumen #93: Fasilitasi Evaluasi Perkembangan Desa serta Lomba Desa dan Kelurahan
Similaritas: 0.17779946327209473

Dokumen #146: Pengelolaan Penerbitan Sertifikasi Keamanan Produk Hewan
Similaritas: 0.17628705501556396

Dokumen #121: Pemeliharaan Peralatan dan Mesin Lainnya
Similaritas: 0.17524303495883942

Dokumen #47: Penetapan Cagar Budaya
Similaritas: 0.17495396733283997



In [41]:
test_doc = word_tokenize("Daya saing perekonomian".lower())
test_doc_vector = model.infer_vector(test_doc)
similar_docs3 = model.docvecs.most_similar(positive = [test_doc_vector])

for doc_id, similarity in similar_docs3:
    print(f'Dokumen #{doc_id}: {df["program"][doc_id]}')
    print(f'Similaritas: {similarity}\n')

Dokumen #77: Pencarian, Pertolongan dan Evakuasi Korban Bencana
Similaritas: 0.08423354476690292

Dokumen #39: Penyediaan dan Pemeliharaan Sarana dan Prasarana Taman Budaya
Similaritas: 0.0752975270152092

Dokumen #7: Pembangunan Ruang Orientasi dan Mobilitas
Similaritas: 0.0731801688671112

Dokumen #114: Penyelenggaraan
Similaritas: 0.06958501785993576

Dokumen #152: Penyediaan Peralatan Rumah Tangga
Similaritas: 0.06500815600156784

Dokumen #50: Pemanfaatan Cagar Budaya
Similaritas: 0.06450675427913666

Dokumen #12: Rehabilitasi Ruang Guru/Kepala Sekolah/TU
Similaritas: 0.06390371918678284

Dokumen #90: Fasilitasi Pembangunan Kawasan Perdesaan
Similaritas: 0.06265239417552948

Dokumen #169: Pelayanan dan Konsultasi Pajak Daerah
Similaritas: 0.06220578774809837

Dokumen #10: Pembangunan Ruang Bina Pribadi dan Sosial untuk Tunalaras (E)
Similaritas: 0.0611494779586792



In [42]:
test_doc = word_tokenize("Lingkungan hidup, ketahanan bencana, dan perubahan iklim".lower())
test_doc_vector = model.infer_vector(test_doc)
similar_docs4 = model.docvecs.most_similar(positive = [test_doc_vector])

for doc_id, similarity in similar_docs4:
    print(f'Dokumen #{doc_id}: {df["program"][doc_id]}')
    print(f'Similaritas: {similarity}\n')

Dokumen #68: Survey Kondisi Jalan/Jembatan
Similaritas: 0.10255845636129379

Dokumen #36: PROGRAM PENGEMBANGAN KEBUDAYAAN
Similaritas: -0.17298470437526703

Dokumen #21: Pembinaan Minat, Bakat dan Kreativitas Siswa
Similaritas: -0.17341920733451843

Dokumen #7: Pembangunan Ruang Orientasi dan Mobilitas
Similaritas: -0.17501473426818848

Dokumen #1: Pembangunan Unit Sekolah Baru (USB)
Similaritas: -0.2135339081287384

Dokumen #55: Penyediaan Peralatan dan Perlengkapan Kantor
Similaritas: -0.21718305349349976

Dokumen #155: Fasilitasi Kunjungan Tamu
Similaritas: -0.22427396476268768

Dokumen #5: Pembangunan Kantin Sekolah
Similaritas: -0.2314145565032959

Dokumen #168: Penetapan Wajib Pajak Daerah
Similaritas: -0.2383086383342743

Dokumen #99: Fasilitasi Pengembangan Inovasi Desa
Similaritas: -0.24251727759838104



In [44]:
test_doc = word_tokenize("Reformasi birokrasi dan tata kelola pemerintahan".lower())
test_doc_vector = model.infer_vector(test_doc)
similar_docs5 = model.docvecs.most_similar(positive = [test_doc_vector])

for doc_id, similarity in similar_docs5:
    print(f'Dokumen #{doc_id}: {df["program"][doc_id]}')
    print(f'Similaritas: {similarity}\n')

Dokumen #36: PROGRAM PENGEMBANGAN KEBUDAYAAN
Similaritas: 0.09360348433256149

Dokumen #155: Fasilitasi Kunjungan Tamu
Similaritas: 0.028565727174282074

Dokumen #68: Survey Kondisi Jalan/Jembatan
Similaritas: -0.023151956498622894

Dokumen #21: Pembinaan Minat, Bakat dan Kreativitas Siswa
Similaritas: -0.037746407091617584

Dokumen #97: Fasilitasi Penetapan dan Penegasan Batas Desa
Similaritas: -0.1033613383769989

Dokumen #75: Gladi Kesiapsiagaan Terhadap Bencana
Similaritas: -0.11443585157394409

Dokumen #64: Pemeliharaan Peralatan dan Mesin Lainnya
Similaritas: -0.11505433171987534

Dokumen #94: Fasilitasi Pelaksanaan Profil Desa dan Kelurahan
Similaritas: -0.1163034439086914

Dokumen #5: Pembangunan Kantin Sekolah
Similaritas: -0.12162772566080093

Dokumen #67: Pembinaan dan Pengawasan Sumber Daya Manusia Kesehatan
Similaritas: -0.12279531359672546



In [49]:
from sentence_transformers import SentenceTransformer
sbert_model = SentenceTransformer('firqaaa/indo-sentence-bert-base')

In [50]:
sentences1 = tagged_data #['Pemeliharaan/Rehabilitasi Sarana dan Prasarana Pendukung Gedung Kantor atau Bangunan Lainnya', 'Penyediaan Jasa Pemeliharaan, Biaya Pemeliharaan, Pajak dan Perizinan Kendaraan Dinas Operasional atau Lapangan']
sentence_embeddings = sbert_model.encode(sentences1)

In [53]:
query = 'Daya saing Sumber Daya Manusia'
query_vec = sbert_model.encode([query])[0]

In [54]:
for sent in sentences1:
    sim = cosine(query_vec, sbert_model.encode([sent])[0])
    print("Sentence = ", sent, "; similarity = ", sim)

Sentence =  TaggedDocument(['Koordinasi,', 'Perencanaan,', 'Supervisi', 'dan', 'Evaluasi', 'Layanan', 'di', 'Bidang', 'Pendidikan'], [0]) ; similarity =  0.23175544
Sentence =  TaggedDocument(['Pembangunan', 'Unit', 'Sekolah', 'Baru', '(USB)'], [1]) ; similarity =  0.25290543
Sentence =  TaggedDocument(['Penambahan', 'Ruang', 'Kelas', 'Sekolah'], [2]) ; similarity =  0.15659615
Sentence =  TaggedDocument(['Pembangunan', 'Ruang', 'Serba', 'Guna/Aula'], [3]) ; similarity =  0.16167128
Sentence =  TaggedDocument(['Pembangunan', 'Sarana,', 'Prasarana', 'dan', 'Utilitas', 'Sekolah'], [4]) ; similarity =  0.31212536
Sentence =  TaggedDocument(['Pembangunan', 'Kantin', 'Sekolah'], [5]) ; similarity =  0.33750397
Sentence =  TaggedDocument(['Pembangunan', 'Rumah', 'Dinas', 'Kepala', 'Sekolah,', 'Guru,', 'Penjaga', 'Sekolah'], [6]) ; similarity =  0.24138534
Sentence =  TaggedDocument(['Pembangunan', 'Ruang', 'Orientasi', 'dan', 'Mobilitas'], [7]) ; similarity =  0.2652955
Sentence =  TaggedDoc

Sentence =  TaggedDocument(['Pemeliharaan', 'Berkala', 'Jalan'], [69]) ; similarity =  0.090377696
Sentence =  TaggedDocument(['Pemeliharaan', 'Rutin', 'Jalan'], [70]) ; similarity =  0.09093028
Sentence =  TaggedDocument(['Pemeliharaan', 'Rutin', 'Jembatan'], [71]) ; similarity =  0.14071065
Sentence =  TaggedDocument(['Sosialisasi,', 'Komunikasi,', 'Informasi', 'dan', 'Edukasi', '(KIE)', 'Rawan', 'Bencana', 'Provinsi', '(Per', 'Jenis', 'Bencana'], [72]) ; similarity =  0.17045732
Sentence =  TaggedDocument(['Pelatihan', 'Pencegahan', 'dan', 'Mitigasi', 'Bencana'], [73]) ; similarity =  0.22451635
Sentence =  TaggedDocument(['Penguatan', 'Kapasitas', 'Kawasan', 'untuk', 'Pencegahan', 'dan', 'Kesiapsiagaan', 'Bencana'], [74]) ; similarity =  0.28217554
Sentence =  TaggedDocument(['Gladi', 'Kesiapsiagaan', 'Terhadap', 'Bencana'], [75]) ; similarity =  0.2542306
Sentence =  TaggedDocument(['Respon', 'Cepat', 'Penanganan', 'Darurat', 'Bencana'], [76]) ; similarity =  0.11442338
Sentence =

Sentence =  TaggedDocument(['Pemberian', 'Bimbingan', 'Peningkatan', 'Produksi', 'Benih/Bibit', 'Ternak', 'dan', 'Tanaman', 'Pakan', 'Ternak,', 'Bahan', 'Pakan,', 'serta', 'Pakan', 'Kewenangan', 'Provinsi'], [134]) ; similarity =  0.29544547
Sentence =  TaggedDocument(['Pengembangan', 'dan', 'Pelaksanaan', 'Sistem', 'Manajemen', 'Mutu', 'Benih/Bibit', 'Ternak', 'dan', 'Tanaman', 'Pakan', 'Ternak,', 'Bahan', 'Pakan,', 'serta', 'Pakan', 'Kewenangan', 'Provinsi'], [135]) ; similarity =  0.37479553
Sentence =  TaggedDocument(['Pengembangan', 'Kapasitas', 'Petugas', 'Pengawas', 'Mutu', 'Benih/Bibit', 'Ternak', 'dan', 'Tanaman', 'Pakan', 'Ternak,', 'Bahan', 'Pakan,', 'serta', 'Pakan', 'Kewenangan', 'Provinsi'], [136]) ; similarity =  0.40393496
Sentence =  TaggedDocument(['Penjaminan', 'Peredaran', 'Benih/Bibit', 'Ternak', 'dan', 'HPT,', 'Bahan', 'Pakan,', 'Pakan'], [137]) ; similarity =  0.37859204
Sentence =  TaggedDocument(['Perencanaan', 'Pengembangan', 'Prasarana,', 'Kawasan', 'dan', 'K

In [55]:
query1 = 'Daya saing perekonomian'
query_vec1 = sbert_model.encode([query])[0]

In [56]:
for sent in sentences1:
    sim = cosine(query_vec1, sbert_model.encode([sent])[0])
    print("Sentence = ", sent, "; similarity = ", sim)

Sentence =  TaggedDocument(['Koordinasi,', 'Perencanaan,', 'Supervisi', 'dan', 'Evaluasi', 'Layanan', 'di', 'Bidang', 'Pendidikan'], [0]) ; similarity =  0.23175544
Sentence =  TaggedDocument(['Pembangunan', 'Unit', 'Sekolah', 'Baru', '(USB)'], [1]) ; similarity =  0.25290543
Sentence =  TaggedDocument(['Penambahan', 'Ruang', 'Kelas', 'Sekolah'], [2]) ; similarity =  0.15659615
Sentence =  TaggedDocument(['Pembangunan', 'Ruang', 'Serba', 'Guna/Aula'], [3]) ; similarity =  0.16167128
Sentence =  TaggedDocument(['Pembangunan', 'Sarana,', 'Prasarana', 'dan', 'Utilitas', 'Sekolah'], [4]) ; similarity =  0.31212536
Sentence =  TaggedDocument(['Pembangunan', 'Kantin', 'Sekolah'], [5]) ; similarity =  0.33750397
Sentence =  TaggedDocument(['Pembangunan', 'Rumah', 'Dinas', 'Kepala', 'Sekolah,', 'Guru,', 'Penjaga', 'Sekolah'], [6]) ; similarity =  0.24138534
Sentence =  TaggedDocument(['Pembangunan', 'Ruang', 'Orientasi', 'dan', 'Mobilitas'], [7]) ; similarity =  0.2652955
Sentence =  TaggedDoc

Sentence =  TaggedDocument(['Pemeliharaan', 'Rutin', 'Jalan'], [70]) ; similarity =  0.09093028
Sentence =  TaggedDocument(['Pemeliharaan', 'Rutin', 'Jembatan'], [71]) ; similarity =  0.14071065
Sentence =  TaggedDocument(['Sosialisasi,', 'Komunikasi,', 'Informasi', 'dan', 'Edukasi', '(KIE)', 'Rawan', 'Bencana', 'Provinsi', '(Per', 'Jenis', 'Bencana'], [72]) ; similarity =  0.17045732
Sentence =  TaggedDocument(['Pelatihan', 'Pencegahan', 'dan', 'Mitigasi', 'Bencana'], [73]) ; similarity =  0.22451635
Sentence =  TaggedDocument(['Penguatan', 'Kapasitas', 'Kawasan', 'untuk', 'Pencegahan', 'dan', 'Kesiapsiagaan', 'Bencana'], [74]) ; similarity =  0.28217554
Sentence =  TaggedDocument(['Gladi', 'Kesiapsiagaan', 'Terhadap', 'Bencana'], [75]) ; similarity =  0.2542306
Sentence =  TaggedDocument(['Respon', 'Cepat', 'Penanganan', 'Darurat', 'Bencana'], [76]) ; similarity =  0.11442338
Sentence =  TaggedDocument(['Pencarian,', 'Pertolongan', 'dan', 'Evakuasi', 'Korban', 'Bencana'], [77]) ; sim

Sentence =  TaggedDocument(['Pengawasan', 'Mutu,', 'Penyediaan', 'dan', 'Peredaran', 'Benih/Bibit', 'Perkebunan'], [133]) ; similarity =  0.33588147
Sentence =  TaggedDocument(['Pemberian', 'Bimbingan', 'Peningkatan', 'Produksi', 'Benih/Bibit', 'Ternak', 'dan', 'Tanaman', 'Pakan', 'Ternak,', 'Bahan', 'Pakan,', 'serta', 'Pakan', 'Kewenangan', 'Provinsi'], [134]) ; similarity =  0.29544547
Sentence =  TaggedDocument(['Pengembangan', 'dan', 'Pelaksanaan', 'Sistem', 'Manajemen', 'Mutu', 'Benih/Bibit', 'Ternak', 'dan', 'Tanaman', 'Pakan', 'Ternak,', 'Bahan', 'Pakan,', 'serta', 'Pakan', 'Kewenangan', 'Provinsi'], [135]) ; similarity =  0.37479553
Sentence =  TaggedDocument(['Pengembangan', 'Kapasitas', 'Petugas', 'Pengawas', 'Mutu', 'Benih/Bibit', 'Ternak', 'dan', 'Tanaman', 'Pakan', 'Ternak,', 'Bahan', 'Pakan,', 'serta', 'Pakan', 'Kewenangan', 'Provinsi'], [136]) ; similarity =  0.40393496
Sentence =  TaggedDocument(['Penjaminan', 'Peredaran', 'Benih/Bibit', 'Ternak', 'dan', 'HPT,', 'Bahan'